In [1]:
### Script to integrate, preprocess and harmonize all available data sources
### In this example
# Single Cell RNA Seq
# Cytokine Data
# Neutrophil Data
# Proteomics

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/jove_r//lib/R/library"


‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.4; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::betwee

In [3]:
source('MS2_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


In [4]:
source('MS1_Functions.r')

###############################################
# Preqrequisites Configurations & Parameters

In [5]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [6]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/input_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/results/


In [7]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [8]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data/"

In [9]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [10]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results/"

In [11]:
## SC Data Configurations

In [12]:
sc_configs = read.csv('configurations/02_Pre_Processing_Configs_SC.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/02_Pre_Processing_Configs_SC.csv'”


In [13]:
head(sc_configs,2)

,configuration_name,data_name,data_type,cell_expr_thres1,cell_expr_thres2,cell_type_exclusion
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,config1,Prepared_sc_Data,h5seurat,50;10,40;20,"Platelet,Plasmablast,pDC,Eryth,Doublet,dnT,cdC1,CD8 TCM,CD8 Proliferating,CD4 Proliferating,NK Proliferating,ASDC,NK_CD56bright,ILC,MAIT,HSPC"


In [14]:
sc_configs = sc_configs[sc_configs$data_name != '',]

In [15]:
## Other Data Configurations

In [16]:
data_configs = read.csv('configurations/02_Pre_Processing_Configs.csv', sep = ',')

In [17]:
data_configs = data_configs[data_configs$configuration_name != '',]   # remove lines with empty configuration names
data_configs = data_configs[!is.na(data_configs$configuration_name),]  # remove lines with NA in configuration name

In [18]:
head(data_configs)

,configuration_name,data_name,file_type,data_type,remove_sample_ids,sample_filtering_thres,feature_filtering_thres,library_adjustment,log_transformation,variable_genes_filtering,quantile_normalization_samples,ribosomal_mitochondrial_gene_filtering,feature_wise_quantile_normalization
,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<lgl>,<lgl>,<lgl>
1,config1,Prepared_Proteomic_Data,csv,proteomic,NA,1.0,0.20,FALSE,FALSE,1.00,FALSE,FALSE,TRUE
2,config1,Prepared_Cytokine_Data,csv,cytokine,NA,1.0,0.20,FALSE,TRUE,1.00,FALSE,FALSE,TRUE
3,config1,Prepared_Clinical_Data,csv,clinical,NA,1.0,0.20,FALSE,TRUE,1.00,FALSE,FALSE,TRUE
4,config1,Prepared_sc_Data,h5seurat,sc,NA,1.0,0.05,TRUE,TRUE,1.00,TRUE,TRUE,TRUE
5,config1,Prepared_Neutrophil_Data,csv,neutrophil,NA,0.9,0.80,TRUE,TRUE,0.75,TRUE,TRUE,TRUE


# Load Data

In [ ]:
### Load sc Data and apply cell-type exclusion per config

In [ ]:
datasets = list()

In [ ]:
## Load Data TABLE FORMAT
if(nrow(sc_configs) > 0){
for(j in 1:nrow(sc_configs)){
    sc_data_name = sc_configs$data_name[j]
    sc_data =  fread(paste0(result_path, '/01_results/01_', sc_data_name, 'Pseudobulk_Table', '.csv'))

    sc_data$V1 = NULL

    ## Split up sc to different configs
    for(i in unique(sc_configs$configuration_name)){    
        for(j in unique(sc_configs$data_name[sc_configs$configuration_name == i])){

            data = sc_data[sc_data$dataset == j,]

            ## Exclude cell-types
            if(!is.na(sc_configs$cell_type_exclusion[sc_configs$configuration_name == i])){
            data = data[!data$type %in% unlist(strsplit(sc_configs$cell_type_exclusion[sc_configs$configuration_name == i] ,',')),]
                }

            datasets[[i]][[j]] = data
            }
        }
    }    
 }   

In [ ]:
### Load the other datasets per config

In [ ]:
for(i in unique(data_configs$configuration_name)){     # for each config
    for(j in unique(data_configs$data_name[data_configs$configuration_name == i])){      # each specifiec data-name
        
        configuration = data_configs[(data_configs$configuration_name == i) & (data_configs$data_name == j),]
        
        if(configuration$file_type == 'csv'){
        data = read.csv(paste0(data_path, j, '.csv'))
        data$X = NULL
        data = melt(data, id.vars = 'sample_id')
        data$dataset = j
        data$type = configuration$data_type
        
        datasets[[i]][[j]] = data
        }
        }
    }

In [ ]:
head(data,2)

In [ ]:
str(datasets)

In [ ]:
data_backup = datasets

In [ ]:
datasets = data_backup

In [ ]:
sort(unique(datasets[[1]][['Prepared_sc_Data']]$type))

# Processing of each data type according to config

## Sample Filter

In [ ]:
head(data_configs,2)

In [ ]:
for(i in 1:nrow(data_configs)){
    ### Remove samples based on specified samples in remove_sample_ids column
    if( (!is.na(data_configs$remove_sample_ids[i])) & (data_configs$remove_sample_ids[i] != '')){
        
        print(paste0('Filtered specific samples for ',data_configs$data_name[i], ' ',  unique( unlist(strsplit(data_configs$remove_sample_ids[i], ',')))))
        
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
        
        ### remove samples
        data = data[! data$sample_id %in% unlist(strsplit(data_configs$remove_sample_ids[i], ',')),]  # TBD check!
        
        ### replace adjusted data
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = data
        
        }
    
     ### Remove samples based on threshold in sample_filtering_thres
     if ( (as.numeric(data_configs$sample_filtering_thres[i]) < 1) & (as.numeric(data_configs$sample_filtering_thres[i]) > 0)){
         
         print(paste0('Filtered samples based on threshold for ',data_configs$data_name[i])) 
         data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
         print(paste0('Amount samples before filtering ', length(unique(data$sample_id))))
         
         ### calculate percentage of features with zero values
         data = data %>% group_by(sample_id, type) %>% mutate(zero_expression_percentage = sum(value == 0)/ n())
         ### filter out samples if percentage higher than threshold
         data = data[data$zero_expression_percentage < data_configs$sample_filtering_thres[i],]
         print(paste0('Amount samples after filtering ', length(unique(data$sample_id))))
         
         ### remove generated columns
         data$zero_expression_percentage = NULL
         
         ### replace adjusted data
         datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = data            
    } 
}
        

## Feature Removal (based on sample expression)

In [ ]:
## Filter out features that are not expressed in a certain amount of sample

In [ ]:
head(data_configs,2)

In [ ]:
data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]

In [ ]:
head(data,2)

In [ ]:
for(i in 1:nrow(data_configs)){

    if( (!is.na(data_configs$feature_filtering_thres[i])) & (data_configs$feature_filtering_thres[i] != '')  & (data_configs$feature_filtering_thres[i] > 0)){
        
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
        
        print(paste0(data_configs$configuration_name[i], ' ' ,data_configs$data_name[i]))
        
        ## Determine data to filter
        data$expression = TRUE
        data$expression[data$value == 0] = FALSE
        expression_filter = data %>% group_by(type, variable) %>% summarise(perc_expression = sum(expression)  )
        expression_filter$perc_expression = expression_filter$perc_expression / length(unique(data$sample_id))
        
        ## Apply filter
        filtered_out = expression_filter[expression_filter$perc_expression <= data_configs$feature_filtering_thres[i],]
        print(paste0( 'Filtered: ' ))
        if(nrow(filtered_out) > 0){
            print(filtered_out %>% dplyr::group_by(type) %>% dplyr::count())
            }
        expression_filter = expression_filter[expression_filter$perc_expression >data_configs$feature_filtering_thres[i],]  # kept data
        
        data = merge(data, expression_filter[,c('type', 'variable')], by.x = c('type', 'variable'), by.y = c('type', 'variable'))   # filter the data
        
        ## Remove expression column 
        data$expression = NULL
        
        ## Replace 
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]  = data
        
      }
}

## Library Adjustment

In [ ]:
head(data_configs,2)

In [ ]:
for(i in 1:nrow(data_configs)){
    if((data_configs$library_adjustment[i] == 'TRUE')){
        
        print(paste0('Library Adjustment for ',data_configs$data_name[i]))
        
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]

        ### Calculate scaling factor per sample
        data = data %>% group_by(sample_id,type) %>% mutate(sample_counts = sum(value))
        data = data %>% group_by(type) %>% mutate(mean_sample_counts = mean(sample_counts))
        
        data$scaling_factor = data$sample_counts/ data$mean_sample_counts
        data$scaling_factor[data$scaling_factor == 0] = 1 # avoid dividing by 0; TBD whether to include or exclude samples with only zero counts in a cell-type
        
        ### Apply scaling to counts
        
        data$value = data$value / data$scaling_factor
        
        ### Save transformed data to list
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = data
        
        }
    }
        

## Gene Filtering (according to cells expressing genes - only for sc Data)

In [ ]:
head(sc_configs,2)

In [ ]:
## Load gene filtering information

In [ ]:
gene_expression_info = read.csv(paste0(result_path, '/01_results/01_Prepared_sc_Data_Gene_Expr_per_Cell_Type.csv'))
gene_expression_info$X = NULL

In [ ]:
head(gene_expression_info,2)

In [ ]:
if(nrow(sc_configs) > 0){
for(i in 1:nrow(sc_configs)){
        
        print(paste0('Gene Filtering for  ',sc_configs$configuration_name[i]))
        
        data = datasets[[sc_configs$configuration_name[i]]][[sc_configs$data_name[i]]]
    
        ## Get thresholds for config
        thres1 = as.numeric(unlist(str_split(sc_configs$cell_expr_thres1[i], ';')))
        thres2 = as.numeric(unlist(str_split(sc_configs$cell_expr_thres2[i], ';')))
        amount_samples = length(unique(data$sample_id))
        print(paste0('Amount Samples', amount_samples))
    
        ## Filter down gene Expression info
        gene_filtering =  gene_expression_info[((gene_expression_info$perc_cells > thres1[1]) & (gene_expression_info$total_amount_cells_expressing_gene > amount_samples * thres1[2])) |
         ((gene_expression_info$perc_cells > thres2[1]) & (gene_expression_info$total_amount_cells_expressing_gene > amount_samples * thres2[2])) ,]
    
        ## Apply to thresholds
        filtered_data = data.frame()
        for( k in unique(data$type)){
            data_cluster = data[(data$type == k) & (data$variable %in% gene_filtering$gene[gene_filtering$cluster == k]),]
            filtered_data = rbind(filtered_data, data_cluster)
            }
        datasets[[sc_configs$configuration_name[i]]][[sc_configs$data_name[i]]] = filtered_data
    }
    }

In [ ]:
### Amount of genes after filtering

In [ ]:
#unique(datasets[[1]][['Prepared_sc_Data']][,c('type', 'variable')])%>% group_by(type) %>% dplyr::count()

## Log Transformation

In [ ]:
head(data_configs,2)

In [ ]:
for(i in 1:nrow(data_configs)){
    if((data_configs$log_transformation[i] == 'TRUE')){
        
        print(paste0('Log Transformation for ',data_configs$data_name[i]))
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
        
        data$value = log2(data$value + 1)  # add pseudocount of 1
        
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = data # save adjusted data
        }
    }
        
        

## Variable Gene Filtering

In [ ]:
head(data_configs,2)

In [ ]:
for(i in 1:nrow(data_configs)){
    ### Filter genes with lowest variance
    if ( (as.numeric(data_configs$variable_genes_filtering[i]) < 1) & (as.numeric(data_configs$variable_genes_filtering[i]) > 0)){
        print(paste0('Variable Genes Filtering for ',data_configs$data_name[i]))
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
        
        ### Calculate variance and threshold
        data = data %>% group_by(variable, type) %>% mutate(feature_variance = var(value)) # variance
        data = data %>% group_by(type) %>% mutate(variance_threshold = quantile(feature_variance, probs = seq(0, 1, 0.01), na.rm = FALSE,
         names = TRUE)[(1-as.numeric(data_configs$variable_genes_filtering[i]))*100])   # threshold
        
        ### Filter
        data = data[data$feature_variance > data$variance_threshold,]
        
        ### remove generated columns
        data$feature_variance = NULL
        data$variance_threshold = NULL
        
        ### Save transformed data to list
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = data
        
        }
    }
        

## Sample Quantile Normalization

In [ ]:
head(data_configs,2)

In [ ]:
for(i in 1:nrow(data_configs)){
    if((data_configs$quantile_normalization_samples[i] == 'TRUE')){
        
        print(paste0('Sample Quantile Normalization for ',data_configs$data_name[i]))
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
        transformed_data = data.frame()
        
        for(k in unique(data$type)){
            data_type = data[data$type == k,]
            data_type = data_type %>% dcast(variable ~ sample_id, value.var = 'value')
            features = data_type$variable
            rownames(data_type) = features
            data_type$variable = NULL
            data_type = data_type[,colSums(is.na(data_type)) != nrow(data_type)] # remove na samples
            data_type  = quantile_normalization(data_type ) 
            data_type = data.frame(data_type)
            data_type$variable = features
            data_type = melt(data_type)
            colnames(data_type) = c('variable', 'sample_id', 'value')
            
            data_type$type = k 
            data_type$dataset = data_configs$data_name[i]
            transformed_data = rbind(transformed_data, data_type)
            
            }
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = transformed_data
        }
    }
            
        
        
        

## Gene Removal (ribosomal, mitochondrial)

In [ ]:
head(data_configs,2)

In [ ]:
for(i in 1:nrow(data_configs)){
    if((data_configs$ribosomal_mitochondrial_gene_filtering[i] == 'TRUE')){
        
        print(paste0('Remove ribosomal and mitochondrial genes for ',data_configs$data_name[i]))
        data = datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]]
        
        ## Remove ribosomal and mitochondiral genes
        data = data[is.na(str_extract(data$variable, '^MT.*|^RPL.*|^RPS.*')),]
        
        datasets[[data_configs$configuration_name[i]]][[data_configs$data_name[i]]] = data
        }
    }
        

        
        

# Merge data types and process

In [ ]:
#str(datasets)

In [ ]:
### Combine all types into one dataset
datasets = lapply(datasets, function(x){
    data = do.call(rbind, x)
    })

In [ ]:
### Overview amount of features per type/ view for first config

In [ ]:
unique(datasets[[1]][,c('type', 'variable')]) %>% group_by(type) %>% dplyr::count()

# Feature Wise Quantile Normalization

In [ ]:
head(data_configs,2)

In [ ]:
data_configs

In [ ]:
data_configs$feature_wise_quantile_normalization

In [ ]:
for(i in names(datasets)){
    
    data = datasets[[i]]
    data$ident = paste0(data$type, '_0_', data$variable)
    final_data = dcast(data, sample_id ~ ident , value.var = "value") # ! with this merging there might be NA values for some samples on some data types
    rownames(final_data) = final_data$sample_id
    final_data$sample_id = NULL
    
    # Remove samples with only NA's
    data_nas = is.na(final_data)
    rowSums(data_nas)  # TBD maybe plot amount of NA per sample
    keep_samples = names(rowSums(data_nas))[rowSums(data_nas) != ncol(final_data)]
    final_data = final_data[keep_samples,]
    data_nas = data_nas[keep_samples,]
    
    # Feature Wise Quantile Normalization in kind of TRUE value
    if(unique(data_configs$feature_wise_quantile_normalization[data_configs$configuration_name == i]) == 'TRUE'){
        print('Applying Feature Wise Quantile Normalization')
        final_data = apply(final_data, 2,stdnorm)
        final_data = data.frame(final_data)
        final_data[data_nas] = NA
        final_data$sample_id = rownames(final_data)
        data_long = melt(final_data)
        data_long$type = str_extract(data_long$variable, '.*_0_')
        data_long$type  = str_replace(data_long$type , '_0_', '')
        data_long$variable = str_replace(data_long$variable, '.*_0_', '')
        datasets[[i]] = data_long
        
        }
    
    }
        
        
        

In [ ]:
head(datasets[[1]],2)

In [ ]:
str(datasets)

# Save the data

In [ ]:
### Adjust variable names
datasets = lapply(datasets, function(x){
    x$gene = x$variable
    x$variable = paste0(x$type, '__', x$variable)
    return(x)
    })

In [ ]:
for(i in names(datasets)){
    write.csv(datasets[[i]], paste0(result_path, '/02_results/02_Combined_Data_', i, '_INTEGRATED',  '.csv'))
    }

In [ ]:
head(datasets[[1]],2)